In [1]:
"""
Load the pre-trained BERT model and tokenizer using a library such as Hugging Face's Transformers.
"""
import torch
from transformers import AutoTokenizer, AutoModel



tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
"""
Load the text data and split it into sentences using a library such as NLTK.
"""
import nltk

with open("text2.txt", "r") as f:
    transcript = f.read()

sentences = nltk.sent_tokenize(transcript)
sentences

['A Stop on the Salt Route\n1000 B.C.',
 'As they rounded a bend in the path that ran beside the river, Lara recognized the silhouette of a fig tree atop a nearby hill.',
 'The weather was hot and the days were long.',
 'The fig tree was in full leaf, but not yet bearing fruit.',
 'Soon Lara spotted other landmarks—an outcropping of limestone beside the path that had a silhouette like a man’s face, a marshy spot beside the river where the waterfowl were easily startled, a tall tree that looked like a man with his arms upraised.',
 'They were drawing near to the place where there was an island in the river.',
 'The island was a good spot to make camp.',
 'They would sleep on the island tonight.',
 'Lara had been back and forth along the river path many times in her short life.',
 'Her people had not created the path—it had always been there, like the river—but their deerskin-shod feet and the wooden wheels of their handcarts kept the path well worn.',
 'Lara’s people were salt traders, 

In [24]:
"""
Generate BERT embeddings for each sentence by passing them through the pre-trained BERT model.
"""
embeddings = []
for sentence in sentences:
    input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    embeddings.append(last_hidden_states.mean(dim=1).squeeze().detach().numpy())

In [25]:
len(embeddings)

186

In [30]:
len(embeddings[0])

768

In [26]:
"""
Use a clustering algorithm, such as hierarchical clustering or k-means, to group the sentences into clusters based on their embeddings.
"""
from sklearn.cluster import AgglomerativeClustering

clusterer = AgglomerativeClustering(n_clusters=5, affinity='cosine', linkage='average')
clusters = clusterer.fit_predict(embeddings)


/Users/akvelon/opt/anaconda3/envs/TopicSegmentation/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [33]:
len(clusters)

186

In [35]:
"""
Apply heuristics to merge or split the clusters into coherent and meaningful topical segments.
"""
segments = []
prev_cluster = clusters[0]
prev_idx = 0
segment_ids = []

for i in range(1, len(clusters)):
    if clusters[i] != prev_cluster:
        segments.append(" ".join(sentences[prev_idx:i]))
        segment_ids.append((prev_idx, i))
        prev_cluster = clusters[i]
        prev_idx = i

segments.append(" ".join(sentences[prev_idx:]))
segment_ids.append((prev_idx, len(sentences)))


In [36]:
len(segments)

22

In [40]:
from scipy.spatial.distance import cdist
import numpy as np

# Calculate the centroid sentence for each cluster
centroid_sentences = []
for start, end in set(segment_ids):
    sentences_in_cluster = sentences[start: end]
    sentence_embeddings = np.array(embeddings[start: end])
    centroid_idx = cdist([s for s in sentence_embeddings], [sentence_embeddings.mean(axis=0)], 'euclidean').argmin()
    centroid_sentences.append(sentences_in_cluster[centroid_idx])

# Print the topic and its corresponding headline or summary
for i, centroid_sentence in enumerate(centroid_sentences):
    print(f"Topic {i+1}: {centroid_sentence}")

Topic 1: There was a cheer from the group.
Topic 2: When they last passed this way, there had been three rafts, all in good condition, left on the east bank.
Topic 3: The island was like a crib, and the group of hills on the sunrise side of the river were like old women mantled in heavy cloaks gathered to have a look at the baby in the crib—that was how Lara’s father had once described the lay of the land.
Topic 4: The deer released a stifled noise, like a gasp, and stopped moving.
Topic 5: When she came to the metal trader’s camp, she found Tarketios lying in a leafy nook secluded from the others; she recognized him by his brawny silhouette.
Topic 6: “Papa?
Topic 7: “I see it!
Topic 8: Po ran past Larth and Lara.
Topic 9: A Stop on the Salt Route
1000 B.C.
Topic 10: And just before we come to the island, we can see the silhouette of that fig tree up there, along the crest of that hill.”
“Good girl!” said Larth, proud of his daughter’s memory and powers of observation.
Topic 11: He cup